# Lab 8: Remote sensing of Vegetation

**Purpose:** The purpose of this lab is to explore different approaches to monitoring vegetation and extracting vegetation information using Earth Engine. Students will explore calculating Green Vegeation Fraction from datasets and well as extracting time series vegetation information from a collection of datasets.

In [ ]:
# import ee api and geemap package
import ee
import math
import geemap
import pandas as pd
from geemap import colormaps as cmaps

In [ ]:
# try to initalize an ee session
# if not authenticated then run auth workflow and initialize
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

## Calculating GVF

Green Vegetation Fraction (or GVF) is a very commonly used variable for hydrologic and weather modeling activities. GVF is a proxy for vegetation cover and relatively easy to calculate from remote sensing data.

We are going to start big with calculating GVF over the US (you can do global scale but that would take running exports...). There are a couple of different methods for calculating GVF. The traditional approaches used land cover to scale each pixel, however [Jiang et al. (2010)](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2009JD013204) looked at calculating GVF at more regular intervals and used global min/max from. We will use that approach for our example:

In [ ]:
# define a year to process
start_time = "2015-01-01"
end_time = "2016-01-01"

In [ ]:
# get landcover data to calculate min/max ndvi from
modis_lc = (
    ee.ImageCollection("MODIS/006/MCD12Q1")
    .filterDate(start_time, end_time)
    .select(["LC_Type1"], ["IGBP"])
    .first()
)

In [ ]:
# define function for QA bit extraction
def extract_bits(image, start, end=None, new_name=None):
    """Function to convert qa bits to binary flag image

    args:
        image (ee.Image): qa image to extract bit from
        start (int): starting bit for flag
        end (int | None, optional): ending bit for flag, if None then will only use start bit. default = None
        new_name (str | None, optional): output name of resulting image, if None name will be {start}Bits. default = None

    returns:
        ee.Image: image with extract bits
    """

    newname = new_name if new_name is not None else f"{start}_bits"

    if (start == end) or (end is None):
        # perform a bit shift with bitwiseAnd
        return image.select([0], [newname]).bitwiseAnd(1 << start)
    else:
        # Compute the bits we need to extract.
        pattern = 0
        for i in range(start, end):
            pattern += int(math.pow(2, i))

        # Return a single band image of the extracted QA bits, giving the band
        # a new name.
        return image.select([0], [newname]).bitwiseAnd(pattern).rightShift(start)


In [ ]:
# define a funtion to preprocess VIIRS imagery
def preprocess(img):
    """Custom QA masking and NDVI calculation method for VIIRS VNP09GA dataset"""
    cloudMask = extract_bits(
        img.select("QF1"), 2, end=3, new_name="cloud_qa"
    ).lt(1)
    shadowMask = extract_bits(
        img.select("QF2"), 3, new_name="shadow_qa"
    ).Not()
    snowMask = extract_bits(img.select("QF2"), 5, new_name="snow_qa").Not()

    sensorZenith = img.select("SensorZenith").abs().lt(6000)

    mask = cloudMask.And(shadowMask).And(snowMask).And(sensorZenith)

    ndvi = img.normalizedDifference(["I2", "I1"]).rename("ndvi")

    return ndvi.updateMask(mask).copyProperties(img,["system:time_start"])


In [ ]:
# load VIIRS imagery and filter by date
viirs_sr = (
    ee.ImageCollection("NOAA/VIIRS/001/VNP09GA")
    .filterDate(start_time, end_time)
)

In [ ]:
# apply preprocessing
viirs_ndvi = viirs_sr.map(preprocess)

In [ ]:
# visualization for land cover data
igbp_vis = {
  "min": 1.0,
  "max": 17.0,
  "palette": [
    '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'
  ],
};

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.addLayer(modis_lc,igbp_vis, "IGBP Landcover")
Map.addLayer(viirs_ndvi.mean(), {"min": 0, "max": 1,"palette":cmaps.get_palette("YlGn")}, 'NDVI Composite');

Map.addLayerControl()

Map

Now that all of the preprocessing is done, we can start processing for the GVF. First, we define our region that we want to extract the min/max NDVI values from land cover types that represent what we would expect from bare and dense vegetated areas.

In [ ]:
# this loads in a global vector file of countries
# filter by country of interest
region = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(
    ee.Filter.eq("country_na","United States")
)

In [ ]:
ndvi_min = viirs_ndvi.min().updateMask(modis_lc.eq(16)).reduceRegion(
    geometry = region.geometry().bounds(),
    reducer = ee.Reducer.mean(),
    scale = 1000,
    bestEffort=True
)

In [ ]:
ndvi_min.getInfo()

In [ ]:
dense_veg = modis_lc.gte(1).And(modis_lc.lt(6))

ndvi_max = viirs_ndvi.max().updateMask(dense_veg).reduceRegion(
    geometry = region.geometry().bounds(),
    reducer = ee.Reducer.mean(),
    scale = 1000,
    bestEffort=True,
)

In [ ]:
ndvi_max.getInfo()

In [ ]:
def calc_gvf(img):
    min_val = ee.Number(ndvi_min.get("ndvi"))
    max_val = ee.Number(ndvi_max.get("ndvi"))

    gvf = img.expression("(ndvi - min) / (max - min)",{
        "ndvi": img.select("ndvi"),
        "min": min_val,
        "max": max_val
    }).clamp(0,1)

    return gvf.clip(region.geometry().bounds()).rename("gvf").copyProperties(img, ["system:time_start"])


viirs_gvf = viirs_ndvi.map(calc_gvf)

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.addLayer(modis_lc,igbp_vis, "IGBP Landcover")
Map.addLayer(viirs_ndvi.mean(), {"min": 0, "max": 1,"palette":cmaps.get_palette("YlGn")}, 'NDVI Composite');
Map.addLayer(viirs_gvf.mean(), {"min": 0, "max": 1,"palette":cmaps.get_palette("YlGn")}, 'GVF Composite');


Map.addLayerControl()

Map

## Harmonic Regression

Lots of interesting analyses can be done to time series by harnessing the `linearRegression()` reducer.  For example, we can estimate linear trend over time. In this example, we are going to seasonality with a harmonic model. 



In [ ]:
# QA mask function
def l8_preprocess(image):
    #Bits 3, 4, and 5 are cloud shadow, snow, and cloud, respectively.
    cloudShadowBitMask = (1 << 3);
    cloudsBitMask = (1 << 5);
    snowBitMask = (1 << 4);

    #Get the pixel QA band.
    qa = image.select('pixel_qa');

    # apply the bit shift and get binary image of different QA flags
    cloud_shadow_qa = qa.bitwiseAnd(cloudShadowBitMask).eq(0)
    snow_qa = qa.bitwiseAnd(snowBitMask).eq(0)
    cloud_qa = qa.bitwiseAnd(cloudsBitMask).eq(0)

    # combine qa mask layers to one final mask
    mask = cloud_shadow_qa.And(snow_qa).And(cloud_qa)

    ndvi = image.normalizedDifference(["B5","B4"]).rename("ndvi")

    # apply mask and return orignal image
    return image.addBands(ndvi).updateMask(mask);

l8_collection = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
)

l8_ndvi = l8_collection.map(l8_preprocess)

To fit this model to the time series, set one cycle per unit time and use ordinary least squares regression to get the weights for each variable:


In [ ]:
def add_variables(img):
    # Compute time in fractional years since the epoch.
    date = ee.Date(img.get("system:time_start"));
    time = ee.Image(
        date.difference(ee.Date('1970-01-01'), 'year')
    ).float().rename('t')

    # calculate cycle from time
    time_radians = time.multiply(2 * math.pi)

    # calculate the sine/cosine from time
    time_cos = time_radians.cos().rename('cos')
    time_sin = time_radians.sin().rename('sin')

    # Return the image with the added bands.
    return ee.Image.cat([
        img,
        time,
        time_cos,
        time_sin,
        ee.Image.constant(1)
    ]).copyProperties(img,["system:time_start"])


In [ ]:
# add the variables for fitting harmonic model
harmonic_l8 = l8_ndvi.map(add_variables)

In [ ]:
# name of the band variable we want to fit
dependent = ee.String("ndvi")

# name of band variables to predict
independents = ee.List(['constant', 't', 'cos', 'sin']);

Fit the model as with the linear trend, using the `linearRegression()` reducer:

In [ ]:
harmonic_trend = (
    harmonic_l8
    .select(independents.add(dependent))
    # The output of this reducer is a 4x1 array image.
    .reduce(ee.Reducer.linearRegression(
        numX = independents.length(), 
        numY = 1
    ))
)


In [ ]:
# Turn the array image into a multi-band image of coefficients.
harmonic_trend_coefficients = (
    harmonic_trend.select('coefficients')
    .arrayProject([0])
    .arrayFlatten([independents])
)


In [ ]:
# define function to apply regression
def apply_harmonic_regression(img):
    y_hat = (
        img.select(independents)
        .multiply(harmonic_trend_coefficients)
        .reduce('sum')
        .rename('fitted')
    )
    return img.addBands(y_hat)


# Compute fitted values.
fitted_harmonic = harmonic_l8.map(apply_harmonic_regression)

Get a time series of observed and fitted NDVI values to see the results:

In [ ]:
pt = ee.Geometry.Point((360+-254.3301), 10.5816)

In [ ]:
def get_timeseries(collection,pt,scale):
    result = collection.getRegion(pt,scale).getInfo()
    df = pd.DataFrame(result[1:])
    df.columns = result[0]
    df["date"]= pd.to_datetime([t['value']*1e6 if type(t)==dict else t*1e6 for t in df["time"]] )
    df.index = df.date
    return df

In [ ]:
timeseries_df = get_timeseries(fitted_harmonic, pt, 30)

In [ ]:
timeseries_df[["ndvi","fitted"]].plot(marker="o",figsize=(10,5))

Although any coefficients can be visualized on a map directly, it is useful and interesting to map the phase and amplitude of the estimated harmonic model.  First, compute phase and amplitude from the coefficients:


In [ ]:
# Compute phase and amplitude.
phase = (
    harmonic_trend_coefficients.select('sin')
    .atan2(harmonic_trend_coefficients.select('cos'))
    # Scale to [0, 1] from radians.
    .unitScale(-math.pi, math.pi)
)

amplitude = (
    harmonic_trend_coefficients.select('sin')
    .hypot(harmonic_trend_coefficients.select('cos'))
    # Add a scale factor for visualization.
    .multiply(3)
)

In [ ]:
# Compute the mean NDVI.
mean_ndvi = l8_ndvi.select('ndvi').mean();

Combine the bands into one image and convert to RGB image:

In [ ]:
# Use the HSV to RGB transform to display phase and amplitude.
rgb = ee.Image.cat([
  phase,      # hue
  amplitude,  # saturation (difference from white)
  mean_ndvi    # value (difference from black)
]).hsvToRgb()


In [ ]:
# Visualize the results
Map = geemap.Map()

Map.setCenter(-121.272, 37.95, 11) # Stockton, CA

Map.addLayer(rgb, {}, 'phase (hue), amplitude (sat), ndvi (val');

Map.addLayerControl()

Map

## Computing peak NDVI

Another very helpful variable to calculate is the time of peak for NDVI. This can be used for crop monitoring and understanding dynamics of different vegetation and land cover types.

In [ ]:
# define a year of interes
yr = 2019

fitted_harmonic_yr = fitted_harmonic.filterDate(f"{yr}-01-01",f"{yr+1}-01-01")

peak_obs = fitted_harmonic_yr.select(["ndvi","t"]).qualityMosaic("ndvi").subtract(yr-1970).multiply(365)

peak_fit = fitted_harmonic_yr.select(["fitted","t"]).qualityMosaic("fitted").subtract(yr-1970).multiply(365)

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.setCenter(-121.272, 37.95, 11) # Stockton, CA

Map.addLayer(peak_obs, {"bands":"t","min":0,"max":365, "palette":cmaps.get_palette("twilight")}, 'Observed peak time (DOY)');
Map.addLayer(peak_fit, {"bands":"t","min":0,"max":365, "palette":cmaps.get_palette("twilight")}, 'Fitted peak time (DOY)');

Map.addLayerControl()

Map